In [2]:
import random
import numpy as np
import sys

#To do:
    #Remove PVectors
    #Rewrite using numpy arrays (to take advantage of, for example, np.linalg.norm)
    #Keep track of loopno

def mag(n):
    summation = 0
    for i in n:
        summation += i ** 2
    return sqrt(summation)

#Initial Conditions -------------------------------------------------
G = 1
timestep = 2
time = 0 #Keep this as 0.

adaptive_timestep = 0

positions, masses, velocities, v_half = [], [], [], []

In [22]:
#Setup stuff

kinetic, potential = 0,0 #Energy
#Non-do_random() case.
if len(masses) != 0:
    accelerations = [[]]*len(masses)

do_random = 1
if do_random == 1:
    stars = 5
    position_range = 200
    velocity_range = 0.3
    mass_range = 0
    accelerations = [[]]*stars

t_squared = timestep**2 #Slight optimization
loopno = 0 #Keep this for later

#Time optimizations
rand = random.uniform
wholelist = range(len(positions))

if do_random == 1:
    initializeRandom()
f = open('data.txt','w') #Clears data file for our energy graphs/ensures there is one
f.close()

print(stars)
print(positions)
print(masses)
print(velocities)
print(accelerations)

5
[[75.00115089119367, 108.40533983581156, 178.25772027375808], [194.21381513557338, 120.4246369163713, -133.77039437676223], [135.19593181998567, -84.68124344141845, -46.60524337201082], [184.001704730187, 196.10916685353243, -66.5132219686192], [141.40761970954162, 85.70475306757282, -187.062643897196]]
[1.0, 1.0, 1.0, 1.0, 1.0]
[[-0.23786107281149835, 0.22474657761928368, -0.0008987239697287319], [0.22232491536930815, -0.24498238723425692, -0.03771825972232634], [-0.13616666472140687, 0.19609743146217712, 0.23885387116194384], [-0.0020761085286605807, 0.02054634537441763, -0.28776358404740515], [-0.22981457812978012, -0.004474743440068429, -0.061044881493373254]]
[[], [], [], [], []]


In [ ]:
while True: #Figure out better mechanism for exit after this
    findEnergy() #Push out kinetic and potential energies into text file
    
    if adaptive_timestep == 1:
        mindist = updateAcceleration() #Finding a(i)
        
        max_acc = max(mag(acceleration1[i]) for i in wholelist)
        dt = sqrt(mindist/max_acc) * 0.05 # force_tol
        updatePosition(dt)
        mindist = updateAcceleration(1) #Finding a(i+1)
        updateVelocity(dt)
    
    else:
        updatePosition(timestep)
        mindist = updateAcceleration()
        updateVelocity(timestep)
        dt = timestep
    
    global time
    time += dt

In [15]:
def initializeRandom():
    #Fixed for Jupyter
    #This sets random values for body positions, masses, velocities, and masses if do_random = 1.
    p = position_range
    v = velocity_range
    m = mass_range
    #Could possible write as a (long) list comprehension
    for i in range(stars):time
        positions.append( [rand(-p,p), rand(-p,p), rand(-p,p)] )
        velocities.append( [rand(-v,v), rand(-v,v), rand(-v,v)] )
        masses.append( 1.0 + rand(-m,m) )

def updateAcceleration():
    #Fixed for Jupyter and KDK
    accelerations = ([0,0,0] for i in accelerations)
    
    mindist = -1
    for initial in range(0,len(masses)-1):
        for affective in range(initial+1,len(masses)):
            distance, xdiff, ydiff, zdiff = findDistance(initial,affective,1) #Find all relevant distances between the two bodies
            
            if distance < mindist or mindist == -1: #Mindist second for optimization (short circuiting)
                mindist = distance
            
            force = G*masses[initial]*masses[affective]/(distance**3) #This should be equal between the two bodies (Newton's 3nd)
            #nca stands for non-component acceleration; it represents the aceleration along the gravitational vector.
            #To turn this into something usable, we then multiply the nca by the previously found x, y, and z differences.
            nca_initial = force/masses[initial] #F = ma ; a = F/m
            nca_affective = force/masses[affective]
            
            accelerations[initial][0] += nca_initial * -xdiff
            accelerations[initial][1] += nca_initial * -ydiff
            accelerations[initial][2] += nca_initial * -zdiff
            
            accelerations[affective][0] += nca_affective * xdiff
            accelerations[affective][1] += nca_affective * ydiff
            accelerations[affective][2] += nca_affective * zdiff
            
    return mindist

def centerofMass(default=0):
    x_upper,y_upper,z_upper,m_lower = 0,0,0,0
    for i in wholelist:
        x_upper += positions[i][0] * masses[i]
        y_upper += positions[i][1] * masses[i]
        z_upper += positions[i][2] * masses[i]
        m_lower += masses[i]
    
    if default != 0:
        return x_upper/m_lower, y_upper/m_lower, z_upper/m_lower

def findEnergy(default='Write'):
    #First kinetic, then potential
    kinetic = 0
    for i in wholelist:
        kinetic += 0.5 * masses[i] * (mag(velocities[i])**2)
    
    potential = 0
    for initial in wholelist:
        for affective in range(initial+1,len(masses)):
            potential += -G * masses[initial]* masses[affective] / findDistance(initial,affective)
    
    #Writing data to file for graphing later
    if default == 'Write':
        f = open('data.txt', 'a')
        f.write('%.5f'%time + ' ' + str(kinetic)+ ' ' + str(potential) + ' ' + str(kinetic+potential) + ' ' + ' '.join('%.5f'%x for x in positions)+' ' + ' '.join('%.5f %.5f %.5f'%(vel[0],vel[1],vel[2]) for vel in velocities)+'\n')
        f.close()
    
def findDistance(a,b,returnDifferences=0): #A and b are indexes in the position array
    #General formula for distance in 3D Space: http://bit.ly/2hnMSWH
    xdiff = positions[a][0] - positions[b][0]
    ydiff = positions[a][1] - positions[b][1]
    zdiff = positions[a][2] - positions[b][2]
    distance = sqrt(xdiff**2 + ydiff**2 + zdiff**2) #Can be replaced with numpy function?
    if returnDifferences == 0:
        return distance
    else:
        return distance, xdiff, ydiff, zdiff

def v_half():
    if adaptive_timestep == 1:
        t_sq = t ** 2
        t = dt
    else:
        t_sq = t_squared
        t = timestep
    
    

def updatePosition(t):
    #Using leapfrog equations (see http://bit.ly/1lrYFR5)
    #Switched succcessfully to KDK. V-half is calculated during position update.
    if adaptive_timestep == 1:
        t_sq = t ** 2
        t = dt
    else:
        t_sq = t_squared
        t = timestep
    
    #Nested for loop list comprehensions?
    for i in wholelist:
        positions[i][0] += (velocities[i][0] + 0.5 * accelerations[i][0]) * t_sq
        positions[i][1] += (velocities[i][1] + 0.5 * accelerations[i][1]) * t_sq
        positions[i][2] += (velocities[i][2] + 0.5 * accelerations[i][2]) * t_sq

def updateVelocity(t):
    if adaptive_timestep == 1:
        t = dt
    else:
        t = timestep
    
    
    for i in wholelist:
        velocities[i][0] += ((velocities[i][0] + 0.5 * accelerations[i][0]) + 0.5 * (accelerations[i][0])) * t
        velocities[i][1] += 0.5 * (acceleration1[i][1] + acceleration2[i][1]) * t
        velocities[i][2] += 0.5 * (acceleration1[i][2] + acceleration2[i][2]) * t